# Resumen

Actualmente, **sin criterios de racionalizacion** para cerrar la brecha costaria **S/ 521,955,198**. Sin embargo si se usan los siguientes **criterios de ordenamiento**:

1) Nombrados excedentes se reubican dentro de su UGEL

2) Las plazas no nombradas excedentes se reubican a nivel nacional para cubrir los requerimientos

Con criterios de **ordenamiento regional** se ahorraria **S/ 58,585,458** y con **ordenamiento nacional** se ahorraria **S/ 65,706,042**


In [147]:
#Libraries
import os 
import pandas as pd
import numpy as np

os.chdir('D:\Brecha-no-docente') # Directorio de trabajo

In [148]:
df = pd.read_stata('Resultados\Brecha con costo.dta')

C:\Users\USUARIO\Anaconda3\lib\site-packages\pandas\io\stata.py:1457: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [149]:
identificacion = ['d_dpto', 'region', 'pliego', 'codue', 'unidadejecutora', 'codooii', 'nombentidad', 'tipo_entidad']
datos = ['pers_limp_mant_n', 'pers_limp_mant_c', 'pers_limp_mant', 'opt_pers_limp_mant', 'exd_pers_limp_mant', 'req_pers_limp_mant',
        'pers_vigilancia_n', 'pers_vigilancia_c', 'pers_vigilancia', 'opt_pers_vigilancia', 'exd_pers_vigilancia', 'req_pers_vigilancia', 
         'coord_adm_ie_n', 'coord_adm_ie_c', 'coord_adm_ie', 'opt_coord_adm_ie', 'exd_coord_adm_ie', 'req_coord_adm_ie',
         'oficinista_n', 'oficinista_c', 'oficinista', 'opt_oficinista', 'exd_oficinista', 'req_oficinista',
         'secretario_n', 'secretario_c', 'secretario', 'opt_secretario', 'exd_secretario', 'req_secretario'
        ]

In [150]:
df_ugel = df.groupby(identificacion)[datos].sum()
df_ugel.reset_index(inplace = True)

for x in datos:
    df_ugel[x] = df_ugel[x].astype(int)

In [151]:
personal = ['pers_limp_mant','pers_vigilancia', 'coord_adm_ie', 'oficinista', 'secretario']

# Desagrego el personal excedente
for x in personal:
    df_ugel[f'exd_{x}_c'] = df_ugel[[f'exd_{x}',f'{x}_c']].min(axis=1)  
    df_ugel[f'exd_{x}_n'] = df_ugel[f'exd_{x}'] - df_ugel[f'exd_{x}_c']

# Brecha sin racionalizacion

En esta brecha se financia todos los requerimientos

In [152]:
# Calculo de la brecha
costo_oficinista = 12*1200+600+12*(1200*0.09)
costo_secretario = 12*1400+600+12*(1400*0.09)
costo_otros      = 12*1150+600+12*(1150*0.09)

brecha_sr = df_ugel.copy()
for x in personal:
    brecha_sr[f'brecha_sr_{x}'] = brecha_sr[f'req_{x}']
    if x == 'oficinista':
        brecha_sr[f'costo_brecha_sr_{x}'] = brecha_sr[f'brecha_sr_{x}']*costo_oficinista
    elif x == 'secretario':
        brecha_sr[f'costo_brecha_sr_{x}'] = brecha_sr[f'brecha_sr_{x}']*costo_secretario
    else:
        brecha_sr[f'costo_brecha_sr_{x}'] = brecha_sr[f'brecha_sr_{x}']*costo_otros


In [153]:
# Region
region_sr = brecha_sr.groupby('d_dpto').sum()

sr_costos = []
sr_personal = []
for x in personal:
    sr_personal.append(f'brecha_sr_{x}')
    sr_costos.append(f'costo_brecha_sr_{x}')

In [154]:
# Brecha en plazas
region_sr[sr_personal].style.format({'brecha_sr_pers_limp_mant'  : '{:20,.0f}', 
                                    'brecha_sr_pers_vigilancia' : '{:20,.0f}', 
                                    'brecha_sr_coord_adm_ie'    : '{:20,.0f}',
                                    'brecha_sr_oficinista'      : '{:20,.0f}', 
                                    'brecha_sr_secretario'      : '{:20,.0f}'})

,brecha_sr_pers_limp_mant,brecha_sr_pers_vigilancia,brecha_sr_coord_adm_ie,brecha_sr_oficinista,brecha_sr_secretario
d_dpto,,,,,
AMAZONAS,71,220,160,39,2
ANCASH,86,503,360,169,16
APURIMAC,51,155,152,37,4
AREQUIPA,127,457,346,83,2
AYACUCHO,70,254,181,90,5
CAJAMARCA,262,580,455,131,11
CALLAO,79,490,210,113,28
CUSCO,269,508,431,113,10
HUANCAVELICA,22,102,104,30,1


In [155]:
# Brecha en costos
region_sr[sr_costos].style.format({'costo_brecha_sr_pers_limp_mant'  : 'S/{:20,.0f}', 
                                    'costo_brecha_sr_pers_vigilancia' : 'S/{:20,.0f}', 
                                    'costo_brecha_sr_coord_adm_ie'    : 'S/{:20,.0f}',
                                    'costo_brecha_sr_oficinista'      : 'S/{:20,.0f}', 
                                    'costo_brecha_sr_secretario'      : 'S/{:20,.0f}'})

,costo_brecha_sr_pers_limp_mant,costo_brecha_sr_pers_vigilancia,costo_brecha_sr_coord_adm_ie,costo_brecha_sr_oficinista,costo_brecha_sr_secretario
d_dpto,,,,,
AMAZONAS,"S/ 1,110,582","S/ 3,441,240","S/ 2,502,720","S/ 635,544","S/ 37,824"
ANCASH,"S/ 1,345,212","S/ 7,867,926","S/ 5,631,120","S/ 2,754,024","S/ 302,592"
APURIMAC,"S/ 797,742","S/ 2,424,510","S/ 2,377,584","S/ 602,952","S/ 75,648"
AREQUIPA,"S/ 1,986,534","S/ 7,148,394","S/ 5,412,132","S/ 1,352,568","S/ 37,824"
AYACUCHO,"S/ 1,094,940","S/ 3,973,068","S/ 2,831,202","S/ 1,466,640","S/ 94,560"
CAJAMARCA,"S/ 4,098,204","S/ 9,072,360","S/ 7,117,110","S/ 2,134,776","S/ 208,032"
CALLAO,"S/ 1,235,718","S/ 7,664,580","S/ 3,284,820","S/ 1,841,448","S/ 529,536"
CUSCO,"S/ 4,207,698","S/ 7,946,136","S/ 6,741,702","S/ 1,841,448","S/ 189,120"
HUANCAVELICA,"S/ 344,124","S/ 1,595,484","S/ 1,626,768","S/ 488,880","S/ 18,912"


In [156]:
# Resumen
resumen_sr_personal = list(brecha_sr[sr_personal].sum())
resumen_sr_costos = list(brecha_sr[sr_costos].sum())
personal_resumen = ['Limpieza y mantenimiento','Vigilantes','Coordinador administrativo','Oficinista','Secretario']
dict_resumen_sr = {'Perfiles': personal_resumen, 'Plazas': resumen_sr_personal, 'Costos': resumen_sr_costos} 
    
resumen_sr = pd.DataFrame(dict_resumen_sr)
resumen_sr.loc['5'] = resumen_sr.sum()
resumen_sr.iloc[5, resumen_sr.columns.get_loc('Perfiles')] = 'Total'

print('En resumen: ')
resumen_sr.style.format({'Plazas':'{:20,.0f}','Costos':'S/{:20,.0f}' })

En resumen: 


,Perfiles,Plazas,Costos
0,Limpieza y mantenimiento,"5,743","S/ 89,832,006"
1,Vigilantes,"14,281","S/ 223,383,402"
2,Coordinador administrativo,"8,355","S/ 130,688,910"
3,Oficinista,"4,236","S/ 69,029,856"
4,Secretario,477,"S/ 9,021,024"
5,Total,"33,092","S/ 521,955,198"


# Brecha con racionalizacion

Criterios de ordenamiento:
1) Los **nombrados excedentes se reubican dentro de su propia UGEL a una IIEE con requerimiento:** El argumento de este movimiento es que al ser IIEE con mas de 140 alumnos son similares en sus caracteristicas

2) **Si aun existen nombrados excedentes ya no se reubican a una IIEE con requerimiento:** Esto se explica porque realizar un moviento dentro de la misma region puede ocasionar gastos de transporte asociados

3) **Las plazas contratadas o vacantes excedentes se mueven a nivel nacional una IIEE con requerimiento**

In [157]:
brecha_cr = df_ugel.copy()

for x in personal:
    # Brecha UGEL
    brecha_cr[f'brecha_ugel_{x}'] = brecha_cr[f'exd_{x}_n'] + brecha_cr[f'exd_{x}_c'] - brecha_cr[f'req_{x}']  
    # Nombrados que no pueden ser reasignados
    brecha_cr[f'nom_exd_rig_{x}'] = brecha_cr[f'exd_{x}_n'] - brecha_cr[f'req_{x}'] 
    brecha_cr[f'nom_exd_rig_{x}'] = np.where(brecha_cr[f'nom_exd_rig_{x}']<0, 0 , brecha_cr[f'nom_exd_rig_{x}'] )

In [158]:
#Brecha regional
for x in personal:
    brecha_cr[f'req1_{x}'] = brecha_cr[f'req_{x}'] - brecha_cr[f'exd_{x}_n'] 
    brecha_cr[f'req1_{x}'] = np.where(brecha_cr[f'req1_{x}']<0,0,brecha_cr[f'req1_{x}'])
    brecha_cr[f'brecha_cr_{x}'] = brecha_cr[f'exd_{x}_c'] - brecha_cr[f'req1_{x}']

region_cr = brecha_cr.groupby('d_dpto').sum()
region_cr[['brecha_cr_pers_limp_mant','brecha_cr_pers_vigilancia','brecha_cr_coord_adm_ie','brecha_cr_oficinista',
           'brecha_cr_secretario']]
for x in personal:
    if x == 'oficinista':
        region_cr[f'costo_brecha_cr_{x}'] = region_cr[f'brecha_cr_{x}']*costo_oficinista
    elif x == 'secretario':
        region_cr[f'costo_brecha_cr_{x}'] = region_cr[f'brecha_cr_{x}']*costo_secretario
    else:
        region_cr[f'costo_brecha_cr_{x}'] = region_cr[f'brecha_cr_{x}']*costo_otros

In [159]:
# Brecha en plazas
list_b_cr = ['brecha_cr_pers_limp_mant','brecha_cr_pers_vigilancia','brecha_cr_coord_adm_ie','brecha_cr_oficinista',
           'brecha_cr_secretario']
region_cr[list_b_cr].style.format({'brecha_cr_pers_limp_mant'  : '{:20,.0f}', 
                                    'brecha_cr_pers_vigilancia' : '{:20,.0f}', 
                                    'brecha_cr_coord_adm_ie'    : '{:20,.0f}',
                                    'brecha_cr_oficinista'      : '{:20,.0f}', 
                                    'brecha_cr_secretario'      : '{:20,.0f}'})

,brecha_cr_pers_limp_mant,brecha_cr_pers_vigilancia,brecha_cr_coord_adm_ie,brecha_cr_oficinista,brecha_cr_secretario
d_dpto,,,,,
AMAZONAS,-32,-220,-153,-29,2
ANCASH,115,-503,-360,-152,3
APURIMAC,-20,-155,-152,-12,2
AREQUIPA,47,-456,-327,-23,11
AYACUCHO,-26,-254,-177,-82,3
CAJAMARCA,-181,-580,-443,-106,4
CALLAO,34,-490,-210,-85,-14
CUSCO,-214,-508,-429,-57,13
HUANCAVELICA,17,-99,-103,-25,8


In [160]:
# Brecha en costos
list_b_cr_costos = ['costo_brecha_cr_pers_limp_mant','costo_brecha_cr_pers_vigilancia','costo_brecha_cr_coord_adm_ie','costo_brecha_cr_oficinista',
           'costo_brecha_cr_secretario']  

for x in list_b_cr_costos:
    region_cr.loc[region_cr[x]>0,x] = 0
    region_cr.loc[region_cr[x]<0,x] = np.absolute(region_cr[x])

region_cr[list_b_cr_costos].style.format({'costo_brecha_cr_pers_limp_mant'  : 'S/{:20,.0f}', 
                                        'costo_brecha_cr_pers_vigilancia' : 'S/{:20,.0f}', 
                                        'costo_brecha_cr_coord_adm_ie'    : 'S/{:20,.0f}',
                                        'costo_brecha_cr_oficinista'      : 'S/{:20,.0f}', 
                                        'costo_brecha_cr_secretario'      : 'S/{:20,.0f}'})

,costo_brecha_cr_pers_limp_mant,costo_brecha_cr_pers_vigilancia,costo_brecha_cr_coord_adm_ie,costo_brecha_cr_oficinista,costo_brecha_cr_secretario
d_dpto,,,,,
AMAZONAS,"S/ 500,544","S/ 3,441,240","S/ 2,393,226","S/ 472,584",S/ 0
ANCASH,S/ 0,"S/ 7,867,926","S/ 5,631,120","S/ 2,476,992",S/ 0
APURIMAC,"S/ 312,840","S/ 2,424,510","S/ 2,377,584","S/ 195,552",S/ 0
AREQUIPA,S/ 0,"S/ 7,132,752","S/ 5,114,934","S/ 374,808",S/ 0
AYACUCHO,"S/ 406,692","S/ 3,973,068","S/ 2,768,634","S/ 1,336,272",S/ 0
CAJAMARCA,"S/ 2,831,202","S/ 9,072,360","S/ 6,929,406","S/ 1,727,376",S/ 0
CALLAO,S/ 0,"S/ 7,664,580","S/ 3,284,820","S/ 1,385,160","S/ 264,768"
CUSCO,"S/ 3,347,388","S/ 7,946,136","S/ 6,710,418","S/ 928,872",S/ 0
HUANCAVELICA,S/ 0,"S/ 1,548,558","S/ 1,611,126","S/ 407,400",S/ 0


## Brecha con ordenamiento regional

In [161]:
#Dataframe Resumen
plazas_cr_regional = []
costos_cr_regional = []

for x in personal:
    # Plazas
    suma_plazas = region_cr.loc[region_cr[f'brecha_cr_{x}']<0, f'brecha_cr_{x}'].sum()
    plazas_cr_regional.append(suma_plazas)
    # Costos
    suma_costos = region_cr.loc[region_cr[f'costo_brecha_cr_{x}']!=0, f'costo_brecha_cr_{x}'].sum()
    costos_cr_regional.append(suma_costos)
    
plazas_cr_regional = list(np.absolute(plazas_cr_regional))
dic_cr_regional = {'Perfiles': personal_resumen, 'Plazas': plazas_cr_regional, 'Costos': costos_cr_regional}
resumen_cr_regional = pd.DataFrame(dic_cr_regional)

#Fila de total
resumen_cr_regional.loc['5'] = resumen_cr_regional.sum()
resumen_cr_regional.iloc[5, resumen_cr_regional.columns.get_loc('Perfiles')] = 'Total'

print('En resumen: ')
resumen_cr_regional.style.format({'Plazas':'{:20,.0f}','Costos':'S/{:20,.0f}' })

En resumen: 


,Perfiles,Plazas,Costos
0,Limpieza y mantenimiento,"3,268","S/ 51,118,056"
1,Vigilantes,"14,275","S/ 223,289,550"
2,Coordinador administrativo,"8,203","S/ 128,311,326"
3,Oficinista,"3,499","S/ 57,019,704"
4,Secretario,192,"S/ 3,631,104"
5,Total,"29,437","S/ 463,369,740"


## Brecha con ordenamiento nacional

In [162]:
#Dataframe Resumen
plazas_cr_nacional = []

for x in personal:
    plazas_cr_nacional.append(f'brecha_cr_{x}')
    
plazas_cr_nacional = list(np.absolute(brecha_cr[plazas_cr_nacional].sum()))
dict_resumen_cr_nacional = {'Perfiles': personal_resumen, 'Plazas': plazas_cr_nacional}
resumen_cr_nacional = pd.DataFrame(dict_resumen_cr_nacional)

#Costos
resumen_cr_nacional['Costos'] = resumen_cr_nacional['Plazas']*costo_otros
resumen_cr_nacional.loc[resumen_cr_nacional['Perfiles'] == 'Oficinista', 'Costos' ] = costo_oficinista*resumen_cr_nacional['Plazas']
resumen_cr_nacional.loc[resumen_cr_nacional['Perfiles'] == 'Secretario', 'Costos' ] = costo_secretario*resumen_cr_nacional['Plazas']

#Fila de total
resumen_cr_nacional.loc['5'] = resumen_cr_nacional.sum()
resumen_cr_nacional.iloc[5, resumen_cr_nacional.columns.get_loc('Perfiles')] = 'Total'

print('En resumen: ')
resumen_cr_nacional.style.format({'Plazas':'{:20,.0f}','Costos':'S/{:20,.0f}' })

En resumen: 


,Perfiles,Plazas,Costos
0,Limpieza y mantenimiento,"2,976","S/ 46,550,592"
1,Vigilantes,"14,275","S/ 223,289,550"
2,Coordinador administrativo,"8,203","S/ 128,311,326"
3,Oficinista,"3,499","S/ 57,019,704"
4,Secretario,57,"S/ 1,077,984"
5,Total,"29,010","S/ 456,249,156"


In [163]:
writer = pd.ExcelWriter('Resultados\Brecha no docente.xlsx', engine='xlsxwriter')

brecha_sr.to_excel(writer, sheet_name='UGEL sin racio')
brecha_cr.to_excel(writer, sheet_name='UGEL con racio')

writer.save()